In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

RAG - Reasoning Engine with Vertex AI Search
@forusone (shins777@gmail.com)


### Reasoning Engine in Vertex AI Search

### Install Vertex AI SDK for Python

In [ ]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    langchain-google-community \
    google-cloud-discoveryengine \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.3/94.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00


In [ ]:
import sys
from IPython.display import Markdown, display

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id="ai-hangsik")

!gcloud config set project ai-hangsik

### Package import and initialize model

In [ ]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Tool,
)

# Preview
from vertexai.preview import (
    rag,
    reasoning_engines)

from vertexai.preview.generative_models import (
    grounding)

# model initialization
vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)
model = GenerativeModel(MODEL)

### Configuration

In [ ]:
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://reasoning_7424"
MODEL = "gemini-1.5-flash"

## RAG Engine Unit Test

### Helper methods

In [ ]:
def create_rag_corpus(display_name:str, path:list):

  # Create RagCorpus
  embedding_model_config = rag.EmbeddingModelConfig(
      publisher_model="publishers/google/models/text-multilingual-embedding-002"
  )

  rag_corpus = rag.create_corpus(
      display_name=display_name,
      embedding_model_config=embedding_model_config,
  )

  # Import Files to the RagCorpus
  rag.import_files(
      rag_corpus.name,
      paths,
      use_advanced_pdf_parsing=True,
      chunk_size=512,  # Optional
      chunk_overlap=100,  # Optional
      max_embedding_requests_per_min=900,  # Optional
  )
  return rag_corpus, rag

#-------------------------------------------------

def retrieve_manual(rag_corpus, rag, query):
  """Manual search for Daou office """

  # Direct context retrieval
  responses = rag.retrieval_query(
      rag_resources=[
          rag.RagResource(
              rag_corpus=rag_corpus.name,
              # Optional: supply IDs from `rag.list_files()`.
              # rag_file_ids=["rag-file-1", "rag-file-2", ...],
          )
      ],
      text= query,
      similarity_top_k=10,  # Optional
      vector_distance_threshold=0.5,  # Optional
  )

  return responses



In [ ]:
display_name = "Daou_manual"

paths = ["gs://daou_office_manual/manual_org/DaouOffice 클라우드 관리자 가이드.pdf",
        "gs://daou_office_manual/manual_org/DaouOffice 클라우드 서비스 가이드 3.5.21.pdf",
        ]

rag_corpus, rag = create_rag_corpus(display_name, paths)

print(rag_corpus)
print(rag)


RagCorpus(name='projects/721521243942/locations/us-central1/ragCorpora/576460752303423488', display_name='Daou_manual', description='', embedding_model_config=EmbeddingModelConfig(publisher_model='projects/ai-hangsik/locations/us-central1/publishers/google/models/text-multilingual-embedding-002', endpoint=None, model=None, model_version_id=None), vector_db=RagManagedDb())
<module 'vertexai.preview.rag' from '/usr/local/lib/python3.10/dist-packages/vertexai/preview/rag/__init__.py'>


In [ ]:

query = "대쉬보드란 무엇인가요?"

responses = retrieve_manual(rag_corpus, rag, query)

for context in responses.contexts.contexts:
    print(context)

    # print(context.text)
    # print(context.distance)

    print('=====')


source_uri: "gs://daou_office_manual/manual_org/DaouOffice 클라우드 관리자 가이드.pdf"
text: "대시<strong>보드"
distance: 0.23968563568667878

=====
source_uri: "gs://daou_office_manual/manual_org/DaouOffice 클라우드 관리자 가이드.pdf"
text: "대시보드"
distance: 0.25012757712787526

=====
source_uri: "gs://daou_office_manual/manual_org/DaouOffice 클라우드 관리자 가이드.pdf"
text: "대시보드 운영자"
distance: 0.25752396520464171

=====
source_uri: "gs://daou_office_manual/manual_org/DaouOffice 클라우드 관리자 가이드.pdf"
text: "동기화 성공 및 실패 여부를 확인하여 실패 시 수정이 필요한 사항을 반영하여 동기화를 진행할 수 있습니다. 6. 메뉴 관리 메뉴관리 \"메뉴관리\"에서는 서비스 페이지에서 제공하는 메뉴를 관리할 수 있습니다. 6.1 홈 서비스 페이지에 보이는 가젯과 위젯 등 대시보드 영역을 관리하는 메뉴 입니다. 대시보드 설정 대시보드 설정에서는 전사 대시보드 운영자를 지정하고, 가젯의 공개 여부를 설정할 수 있습니다. 대시보드 운영자 설정 대시보드 운영자를 지정합니다. 대시보드 운영자는 전사 대시보드를 편집할 수 있는 권한을 가지며, 대시보드 운영자 는 어드민에 접속할 필요없이 웹 서비스에서 바로 전사 대시보드를 편집할 수 있습니다. 1. 사이트 어드민의 [메뉴관리>홈 > 대시보드]을 클릭합니다. 2. 운영자 추가를 클릭합니다. 3. 조직도에서 운영자로 지정할 사용자를 모두 선택합니다. 4. 설정이 완료되면 저장을 클릭합니다."
distance: 0.2

## Integrate with Reasoning engine

### Helper methods

In [ ]:
# Initialize session history
store = {}

def get_session_history(session_id: str):
  """
    Get session history
  """
  if session_id not in store:
      store[session_id] = ChatMessageHistory()
  return store[session_id]

#---------------------------------------------------------------
def interactive_chat(agent, query, session_id):
  """
  Chat with agent with chat history.
  """
  respone = agent.query(input=query,
                        config={"configurable": {"session_id": session_id}},
  )
  return respone['output']

In [ ]:
# Create a RAG retrieval tool
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=rag_corpus.name,  # Currently only 1 corpus is allowed.
                    # Optional: supply IDs from `rag.list_files()`.
                    # rag_file_ids=["rag-file-1", "rag-file-2", ...],
                )
            ],
            similarity_top_k=10,  # Optional
            vector_distance_threshold=0.5,  # Optional
        ),
    )
)

In [ ]:
from googleapiclient import discovery
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain.memory import ChatMessageHistory
from langchain_core import prompts


# Define prompt template
prompt = {
    "history": lambda x: x["history"],
    "input": lambda x: x["input"],
    "agent_scratchpad": (lambda x: format_to_tool_messages(x["intermediate_steps"])),
} | prompts.ChatPromptTemplate.from_messages(
    [
        prompts.MessagesPlaceholder(variable_name="history"),
        ("user", "{input}"),
        prompts.MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:
from vertexai.preview import reasoning_engines

agent = reasoning_engines.LangchainAgent(
    prompt=prompt,
    model=MODEL,
    chat_history=get_session_history,
    model_kwargs={"temperature": 0},
    tools=[rag_retrieval_tool],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [ ]:
from time import perf_counter

session_id = "sess_101"

while True:
  query = input('사용자: ')

  if query == '종료': break

  t1_start = perf_counter()
  output = interactive_chat(agent, query, session_id)
  t1_end = perf_counter()

  display(Markdown(f"{t1_end-t1_start}"))
  display(Markdown(f"AI Agent : {output}"))
  print(f"------------------------------------ ")

chat_history = get_session_history(session_id)
chat_history.clear()
print(chat_history.messages)

사용자: 대쉬보드는 무엇인가요 ?


2.0397631470000306

AI Agent : 대시보드는 서비스 페이지에 보이는 가젯과 위젯 등을 관리하는 메뉴입니다. 대시보드 설정에서는 전사 대시보드 운영자를 지정하고, 가젯의 공개 여부를 설정할 수 있습니다. 


------------------------------------ 
사용자: 가젯의 공개여부는 어떻게 설정하나요 ?


1.2724267519997738

AI Agent : 사이트 어드민의 [메뉴관리 > 홈 > 대시보드]를 클릭하여 대시보드 설정 메뉴로 이동합니다.  대시보드 설정에서 가젯의 공개 여부를 설정할 수 있습니다. 


------------------------------------ 
사용자: 지금 어떤 제품을 이야기하나요 ?


1.413776789000167

AI Agent : DaouOffice 클라우드 관리자 가이드에 따르면, 대시보드는 서비스 페이지에 보이는 가젯과 위젯 등을 관리하는 메뉴입니다. 


------------------------------------ 
사용자: 종료
[]
